In [2]:
import numpy as np
import pandas as pd
import os
import datetime

import pyarrow.parquet as pq

# Go to settings and set files.maxMemoryForLargeFilesMB to 16384 before running this

file_dir = '/Users/zhicong/Desktop/KCL/Ethereum_DEX_Data/dex_swaps'

# Load Data

In [3]:
data = pd.DataFrame(columns = ['BLOCK_NUMBER','BLOCK_TIMESTAMP','TX_HASH','SENDER','TX_TO','PLATFORM','EVENT_NAME','AMOUNT_IN','SYMBOL_IN','AMOUNT_OUT','SYMBOL_OUT'])
parquet_file = pq.ParquetFile(os.path.join(file_dir, "DEXs_swaps_full.gzip"))

# Set tokens that we want to analyze
token1 = "USDC"
token2 = "WETH"

# Read by batch to be able to fit dataframe into memory
for i in parquet_file.iter_batches(batch_size = 1000000):

    # Filter out only tokens we want
    temp = i.to_pandas()
    temp = temp[((temp['SYMBOL_IN'] == token1) & (temp['SYMBOL_OUT'] == token2)) | ((temp['SYMBOL_IN'] == token2) & (temp['SYMBOL_OUT'] == token1))]

    data = pd.concat([data,temp], axis = 0)
    del temp

# Final cleaning
data.reset_index(drop = True, inplace = True)
data['AMOUNT_IN'] = data['AMOUNT_IN'].astype(float)
data['AMOUNT_OUT'] = data['AMOUNT_OUT'].astype(float)
data['BLOCK_TIMESTAMP'] = pd.to_datetime(data['BLOCK_TIMESTAMP'])

# Get swap price of WETH/USDC
data['swap_price'] = np.where(data['SYMBOL_IN'] == 'WETH', data['AMOUNT_OUT']/data['AMOUNT_IN'], data['AMOUNT_IN']/data['AMOUNT_OUT'])
data.replace([np.inf, -np.inf], np.nan, inplace = True)
data.dropna(subset = ['swap_price'], how = "all", inplace = True)

# Remove anomalies (simple as removing obsurb values)
data = data[(data['swap_price'] > 100) & (data['swap_price'] < 5000)]
data = data[~data['PLATFORM'].isin(['curve'])]
data.reset_index(inplace = True, drop = True)

# Add a column called 'volume' that indicates the volume of swap in USDC
data['volume'] = np.where(data['SYMBOL_IN'] == 'WETH', data['AMOUNT_OUT'], data['AMOUNT_IN'])

data

,BLOCK_NUMBER,BLOCK_TIMESTAMP,TX_HASH,SENDER,TX_TO,PLATFORM,EVENT_NAME,AMOUNT_IN,SYMBOL_IN,AMOUNT_OUT,SYMBOL_OUT,swap_price,volume
0,17180769,2023-05-03 14:24:59,0x7d8733aadbce610cfdc1caaa58bcc7d03e82b908aee1...,0xef1c6e67703c7bd7107eed8303fbe6ec2554bf6b,0xde334292a716841e13e8706337d0ada75047e482,uniswap-v3,Swap,0.250000,WETH,463.362810,USDC,1853.451240,463.362810
1,17180769,2023-05-03 14:24:59,0xb192b7f2315e82318a384384531921917cbd626aa198...,0xef1c6e67703c7bd7107eed8303fbe6ec2554bf6b,0xef1c6e67703c7bd7107eed8303fbe6ec2554bf6b,uniswap-v3,Swap,25000.000000,USDC,13.475117,WETH,1855.271421,25000.000000
2,17180768,2023-05-03 14:24:47,0x147efc1e25a23b5d5b11ef5872fe278ce6d54e75853e...,0xef1c6e67703c7bd7107eed8303fbe6ec2554bf6b,0xef1c6e67703c7bd7107eed8303fbe6ec2554bf6b,uniswap-v3,Swap,1500.000000,USDC,0.808523,WETH,1855.233983,1500.000000
3,17180768,2023-05-03 14:24:47,0x44b46be85109b5f480e56620f6a0aae9ff4e5e937950...,0xdef1c0ded9bec7f1a1670819833240f027b25eff,0xe66b31678d6c16e9ebf358268a790b763c133750,sushiswap,Swap,0.000054,WETH,0.099671,USDC,1848.608004,0.099671
4,17180767,2023-05-03 14:24:35,0xb676a40e792c35a264a8115cddb8024b6025d1985495...,0xd1742b3c4fbb096990c8950fa635aec75b30781a,0x94b86ca6f7a495930fe7f552eb9e4cbb5ef2b736,uniswap-v2,Swap,500.000000,USDC,0.268951,WETH,1859.073404,500.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11179463,10045107,2020-05-11 13:08:48,0x840bb5ab0a779b4f733443651ac54dabe24e243c3606...,0x57ead0a9f49fafdd2447f615b036f3c316af5171,0x57ead0a9f49fafdd2447f615b036f3c316af5171,uniswap-v2,Swap,0.001000,WETH,0.204870,USDC,204.870000,0.204870
11179464,10014418,2020-05-06 18:51:12,0x735cf98e86a5df67b6a837ae50de1d7a589d9f6baaf0...,0xf164fc0ec4e93095b804a4795bbe1e041497b92a,0x12ede161c702d1494612d19f05992f43aa6a26fb,uniswap-v2,Swap,0.000689,USDC,0.000003,WETH,201.910785,0.000689
11179465,10013764,2020-05-06 16:24:55,0x43b6bfd06dde0814fe9c1b63ce98ec4c67c72d96169d...,0xf164fc0ec4e93095b804a4795bbe1e041497b92a,0x8688a84fcfd84d8f78020d0fc0b35987cc58911f,uniswap-v2,Swap,0.000005,WETH,0.001000,USDC,200.678676,0.001000
11179466,10008585,2020-05-05 21:12:35,0x697b7aaca56a80a8d3a2f560ed7f1ecb97c22b2edd6e...,0xf164fc0ec4e93095b804a4795bbe1e041497b92a,0x11e4857bb9993a50c685a79afad4e6f65d518dda,uniswap-v2,Swap,0.000049,WETH,0.010000,USDC,202.919065,0.010000


# Address Clustering

Before identifying arbitrageurs, we want to be able to cluster addresses (generated from wallet application) together if they are the same entity.

We simply make the assumption that people only conduct swaps in DEXs and "Sender" and "Tx_to" will always be the same entity.

In [64]:
address_cluster = data[['SENDER','TX_TO']].copy()
address_cluster.drop_duplicates(inplace = True)
address_cluster = address_cluster[address_cluster['SENDER'] != address_cluster['TX_TO']]
address_cluster = pd.DataFrame(address_cluster.groupby('SENDER')['TX_TO'].apply(list).to_dict().items(),
                               columns = ['id','address'])

for i in range(len(address_cluster)):
    address_cluster.loc[i,'address'] = [address_cluster.loc[i,'id']] + address_cluster.loc[i,'address']
    address_cluster.loc[i,'id'] = i

address_cluster

,id,address
0,0,"[0x000000000000006f6502b7f2bbac8c30a3f67e9a, 0..."
1,1,"[0x0000000000000eb4ec62758aae93400b3e5f7f18, 0..."
2,2,"[0x0000000000002d534ff79e9c69e7fcc742f0be83, 0..."
3,3,"[0x00000000000062f06c7007906b2a4034fa5c4818, 0..."
4,4,"[0x000000000000660def84e69995117c0176ba446e, 0..."
...,...,...
1405,1405,"[0xff26d7d5710b916b62c8317c9228c06a38446203, 0..."
1406,1406,"[0xff2fbc735d33ae830f056107f1b551783ec4ed5b, 0..."
1407,1407,"[0xff3d72aef9e537699287b1049b2284510c000000, 0..."
1408,1408,"[0xffcd3404852afb1420540c8465433f832727bdd7, 0..."


Now, we have a dataframe that has the list of addresses that are clustered.

# Identifying Arbitrageurs

We search for pairs of trades that are:
- made by the same address,
- of the same volume but in different exchanges,
- and within 3 blocks.

We first search for addresses that have traded in multiple exchanges.

In [13]:
arbitrageurs = data[(data['SENDER'] == data['TX_TO']) & (data['PLATFORM'].isin(['uniswap-v2','sushiswap']))][['SENDER','PLATFORM']].drop_duplicates()
arbitrageurs = arbitrageurs.groupby(['SENDER']).size().reset_index(name = 'No_of_Exchange')
arbitrageurs = arbitrageurs[arbitrageurs['No_of_Exchange'] >= 2]
arbitrageurs = arbitrageurs['SENDER']
arbitrageurs

0       0x000000000000006f6502b7f2bbac8c30a3f67e9a
3       0x0000000000000eb4ec62758aae93400b3e5f7f18
4       0x00000000000017c75025d397b91d284bbe8fc7f2
6       0x0000000000002d534ff79e9c69e7fcc742f0be83
7       0x0000000000005117dd3a72e64a705198753fdd54
                           ...                    
1970    0xff26d7d5710b916b62c8317c9228c06a38446203
1971    0xff2fbc735d33ae830f056107f1b551783ec4ed5b
1972    0xff40485f9b692b3775edfc1550964e2ff613cbc6
1974    0xffbb58e97332d9244b997ec2b81eff5bf1be576f
1980    0xfffb2ab6f574dd3b2a9661cafb5e60208686ed63
Name: SENDER, Length: 988, dtype: object

In [22]:
arbitrage_trades = data[(data['SENDER'] == data['TX_TO']) & (data['SENDER'].isin(arbitrageurs))].copy()
arbitrage_trades.sort_values(by = ['SENDER','BLOCK_NUMBER'], ascending = False, inplace = True)
arbitrage_trades.head(50)

,BLOCK_NUMBER,BLOCK_TIMESTAMP,TX_HASH,SENDER,TX_TO,PLATFORM,EVENT_NAME,AMOUNT_IN,SYMBOL_IN,AMOUNT_OUT,SYMBOL_OUT,swap_price,volume
8606985,12543172,2021-05-31 16:23:57,0x5f06d3abc9a1d51f19849f6dfd46f1dcf9cdeda66008...,0xfffb2ab6f574dd3b2a9661cafb5e60208686ed63,0xfffb2ab6f574dd3b2a9661cafb5e60208686ed63,sushiswap,Swap,4.806072e+03,USDC,1.795842e+00,WETH,2676.221649,4.806072e+03
8606986,12543172,2021-05-31 16:23:57,0x5f06d3abc9a1d51f19849f6dfd46f1dcf9cdeda66008...,0xfffb2ab6f574dd3b2a9661cafb5e60208686ed63,0xfffb2ab6f574dd3b2a9661cafb5e60208686ed63,sushiswap,Swap,6.421196e+04,USDC,2.400160e+01,WETH,2675.319764,6.421196e+04
8618382,12536963,2021-05-30 17:07:17,0x459447e288e94ccde579576896acba83a16d0e5408c9...,0xfffb2ab6f574dd3b2a9661cafb5e60208686ed63,0xfffb2ab6f574dd3b2a9661cafb5e60208686ed63,sushiswap,Swap,2.910820e+00,WETH,7.000016e+03,USDC,2404.826316,7.000016e+03
8664176,12514741,2021-05-27 06:09:40,0xeb80bb20785ccc9adca1079b3874c791f6719bd73e10...,0xfffb2ab6f574dd3b2a9661cafb5e60208686ed63,0xfffb2ab6f574dd3b2a9661cafb5e60208686ed63,sushiswap,Swap,7.544656e+00,WETH,2.044186e+04,USDC,2709.449359,2.044186e+04
8668703,12512562,2021-05-26 22:17:30,0xaf69766db3e8580a15dd1fad940e5772a20437592dea...,0xfffb2ab6f574dd3b2a9661cafb5e60208686ed63,0xfffb2ab6f574dd3b2a9661cafb5e60208686ed63,uniswap-v2,Swap,1.637039e+03,USDC,5.781349e-01,WETH,2831.586712,1.637039e+03
8668704,12512562,2021-05-26 22:17:30,0xaf69766db3e8580a15dd1fad940e5772a20437592dea...,0xfffb2ab6f574dd3b2a9661cafb5e60208686ed63,0xfffb2ab6f574dd3b2a9661cafb5e60208686ed63,sushiswap,Swap,2.069545e+04,USDC,7.303297e+00,WETH,2833.712699,2.069545e+04
8726622,12490576,2021-05-23 12:42:01,0xd941290d9f139a04fdca2a43e534c83b2ec9ebd7e45f...,0xfffb2ab6f574dd3b2a9661cafb5e60208686ed63,0xfffb2ab6f574dd3b2a9661cafb5e60208686ed63,sushiswap,Swap,2.674074e+01,WETH,5.294999e+04,USDC,1980.124565,5.294999e+04
8730172,12489642,2021-05-23 08:58:57,0xa59747b8be5abbac5b069e0fbcdc1d300d88b2791703...,0xfffb2ab6f574dd3b2a9661cafb5e60208686ed63,0xfffb2ab6f574dd3b2a9661cafb5e60208686ed63,sushiswap,Swap,2.512064e+01,WETH,5.042844e+04,USDC,2007.450115,5.042844e+04
8730209,12489633,2021-05-23 08:57:50,0xd621f6246eae33a943145a815031e5d8e24221c922e9...,0xfffb2ab6f574dd3b2a9661cafb5e60208686ed63,0xfffb2ab6f574dd3b2a9661cafb5e60208686ed63,sushiswap,Swap,9.719491e+00,WETH,1.947853e+04,USDC,2004.069054,1.947853e+04
8730512,12489578,2021-05-23 08:44:14,0x01284a577ababfa3663fc52ddb69363a5bf921e7fe18...,0xfffb2ab6f574dd3b2a9661cafb5e60208686ed63,0xfffb2ab6f574dd3b2a9661cafb5e60208686ed63,sushiswap,Swap,6.348813e+00,WETH,1.314886e+04,USDC,2071.073782,1.314886e+04
